In [1]:
# Gerekli kütüphaneleri yükleyelim
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
import numpy as np

train_path = "C:/Users/ASUS/Desktop/Regression of Used Car Prices/train.csv"

test_path = "C:/Users/ASUS/Desktop/Regression of Used Car Prices/test.csv"

train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)

In [2]:
# 1. Veri Temizliği ve Hazırlama
# Eksik verileri doldurma (örneğin, temiz başlıklar ve iç/dış renkler)
train_data['clean_title'].fillna('Unknown', inplace=True)
test_data['clean_title'].fillna('Unknown', inplace=True)

# Kategorik verileri one-hot encoding ile sayısal değerlere dönüştürme
categorical_columns = ['brand', 'model', 'fuel_type', 'engine', 'transmission', 'ext_col', 'int_col', 'accident', 'clean_title']
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)  # sparse yerine sparse_output kullanılıyor
encoded_train_array = encoder.fit_transform(train_data[categorical_columns])
encoded_test_array = encoder.transform(test_data[categorical_columns])

# Encoder tarafından oluşturulan yeni sütun adlarını alalım
encoded_columns = encoder.get_feature_names_out(categorical_columns)

# Numpy array'lerini DataFrame'e çevirirken sütun adlarını ekleyelim
encoded_train_data = pd.DataFrame(encoded_train_array, columns=encoded_columns, index=train_data.index)
encoded_test_data = pd.DataFrame(encoded_test_array, columns=encoded_columns, index=test_data.index)

# Orijinal veriyle birleştirme
train_data = train_data.join(encoded_train_data).drop(columns=categorical_columns)
test_data = test_data.join(encoded_test_data).drop(columns=categorical_columns)

# 2. Model Seçimi ve Eğitimi
# Bağımlı ve bağımsız değişkenler
X = train_data.drop(columns=['price', 'id'])
y = train_data['price']

# Veri setini eğitim ve doğrulama setlerine ayırma
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

# Modeli tanımlama ve eğitme
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

# Tahminleri yapma
y_pred = model.predict(X_valid)

# Performans değerlendirmesi (RMSE)
rmse = mean_squared_error(y_valid, y_pred, squared=False)
print(f"Validation RMSE: {rmse}")


C:\Users\ASUS\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Validation RMSE: 77655.92998095411


MemoryError: Unable to allocate 3.38 GiB for an array with shape (3613, 125690) and data type float64

In [3]:
# 6. Batch'ler Halinde Test Setinde Tahminler
batch_size = 1000
all_predictions = pd.DataFrame()

for i in range(0, len(test_data), batch_size):
    batch = test_data[i:i+batch_size]
    batch_predictions = model.predict(batch.drop(columns=['id']))
    
    batch_output = pd.DataFrame({'id': batch['id'], 'price': batch_predictions})
    all_predictions = pd.concat([all_predictions, batch_output])

# 7. Sonuçları CSV Dosyasına Kaydetme
all_predictions.to_csv('predictions.csv', index=False)

MemoryError: Unable to allocate 3.38 GiB for an array with shape (3611, 125690) and data type float64